This is the software companion to the paper "Computing Galois groups of Ehrhart polynomials in OSCAR" by [Claus Fieker](https://www.mathematik.uni-kl.de/~fieker/en/publications.html),
[Tommy Hofmann](https://www.thofma.com/) and
[Michael Joswig](https://page.math.tu-berlin.de/~joswig/). 

The functions described will appear in the upcoming version 0.8.2 of OSCAR (to arrive soon).  You can check the code now by cloning the [current master](git@github.com:oscar-system/Oscar.jl.git) and running the commands below.  Set your working directory to the top level of the cloned git repo.

In [1]:
import Pkg; Pkg.add("Revise"); using Revise; Pkg.activate("."); Pkg.instantiate();

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `/cache/polymake/oscar-system/Oscar.jl/Project.toml`
  No Changes to `/cache/polymake/oscar-system/Oscar.jl/Manifest.toml`
  Activating project at `/cache/polymake/oscar-system/Oscar.jl`


In [2]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.8.2-DEV ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2022 by The Oscar Development Team


The Ehrhart polynomial of the cube factors into linear factors, whence the Galois group is trivial.  Here we show it for the 3-cube, but this generalizes to arbitrary dimensions.

In [3]:
C = cube(3)

LC = ehrhart_polynomial(C)
@show factor(LC)

galois_group(LC)

factor(LC) = 1 * (2*x + 1)^3


(Group(()), Galois Context for 2*x + 1 and prime 3)

The Fano simplices exist in all dimensions, they have the origin as the unique interior lattice point.

In [4]:
S = fano_simplex(14)

A polyhedron in ambient dimension 14

The normal toric variety associated with the face fan is smooth; in fact, it is a projective space.

In [5]:
issmooth(NormalToricVariety(face_fan(S)))

true

Here we are interested in the Galois group of the Ehrhart polynomial.  This is our first non-trivial example.

In [6]:
@time G = galois_group(ehrhart_polynomial(S))

  4.682191 seconds (11.27 M allocations: 677.283 MiB, 4.02% gc time, 71.11% compilation time)


(<permutation group of size 645120 with 9 generators>, Galois Context for 15*x^14 + 105*x^13 + 24115*x^12 + 143325*x^11 + 7724717*x^10 + 37312275*x^9 + 725938785*x^8 + 2681453775*x^7 + 21964438496*x^6 + 56663366760*x^5 + 201186840400*x^4 + 310989260400*x^3 + 429952217472*x^2 + 283465647360*x + 87178291200 and prime 59)

The Ehrhart polynomials of the Fano simplices are known.  Here we give a closed form description. 

In [7]:
Qt, t = QQ["t"]

LS(d) = binomial(t+d+1,d+1) - binomial(t,d+1)

LS (generic function with 1 method)

Here is the case $d=6$, normalized to making the coefficients integral.

In [8]:
factorial(6) * LS(6)

7*t^6 + 21*t^5 + 385*t^4 + 735*t^3 + 2128*t^2 + 1764*t + 720

Its roots lie on the *critical line* $\operatorname{Re}(z)=-1/2$.

In [9]:
roots(LS(6), ComplexField(64))

6-element Vector{acb}:
 [-0.50000000000000000000 +/- 1e-25] + [-0.4466484333276384126 +/- 7.42e-20]*im
 [-0.50000000000000000000 +/- 1e-25] + [0.4466484333276384126 +/- 7.42e-20]*im
 [-0.5000000000000000000 +/- 5.43e-20] + [-2.157723183499887922 +/- 3.77e-19]*im
 [-0.50000000000000000000 +/- 1e-25] + [2.157723183499887922 +/- 3.77e-19]*im
 [-0.5000000000000000000 +/- 5.43e-20] + [-6.811368132790436334 +/- 7.10e-19]*im
 [-0.5000000000000000000 +/- 5.43e-20] + [6.811368132790436334 +/- 7.10e-19]*im

In this case the structure of Galois group can be recognized.  Note that one group occurs under many names.

In [10]:
describe(galois_group(LS(6))[1])

"C2 x S4"

OSCAR provides access to the database [polyDB](https://db.polymake.org).  Among other objects it comprises the complete list of smooth Fano polytopes of dimension $\leq 8$.

Notice, however, that OSCAR defines toric varieties from lattice polytopes through normal fans by default.  Hence all "smooth" polytopes are necessarily simple.

Our goal is to provide a tally of orders of Galois groups of Ehrhart polynomials of the polars of all $6$-dimensional polytopes from that collection.

In [11]:
db = Polymake.Polydb.get_db();
collection = db["Polytopes.Lattice.SmoothReflexive"];
query = Dict("DIM"=>6);
results = Polymake.Polydb.find(collection, query);

We collect the polars of those polytopes and their Ehrhart polynomials.

In [12]:
PP6 = [ polarize(Polyhedron(P)) for P in results ]
E6 = [ ehrhart_polynomial(P) for P in PP6 ];

size(E6)

(7622,)

Here is the final statistics.

In [13]:
sort(MSet(order(galois_group(e)[1]) for e = E6).dict)

OrderedCollections.OrderedDict{Any, Int64} with 6 entries:
  4  => 623
  6  => 1
  8  => 22
  12 => 44
  16 => 310
  48 => 6622

Above we mentioned the critical line $\operatorname{Re}(z)=-1/2$.

In [14]:
function on_critical_line(L)    
    for z in roots(L, ComplexField(64))
        if Float64(real(z)) != -1/2
            return false
        end
    end
    return true
end

on_critical_line (generic function with 1 method)

The following duplicates what we already saw above.

In [15]:
on_critical_line(LS(6))

true

Hegedüs and Kasprzyk (2011) showed that there are exactly four types of polars of smooth Fano $6$-polytopes such that the zeros of their Ehrhart polynomials do not lie on the critical line.

Let's find them.

In [16]:
function check_critical_line(EE)
    idx = 1
    non_critical = []
    for e in EE
        if !on_critical_line(e)
            push!(non_critical, idx)
        end
        idx+=1
    end
    return non_critical
end

NC6 = check_critical_line(E6)

4-element Vector{Any}:
  721
  804
 2617
 3155

And these are their Galois groups.  The one with index 3155 is lattice isomorphic to (the polar of) a free sum of del Pezzo polytopes $\operatorname{DP}(2)\oplus\operatorname{DP}(4)$.

It turns out that this is the only (polar of a) smooth Fano polytope whose Ehrhart polynomial has a galois group of order six; check the tally above.  

In [17]:
[ describe(galois_group(E6[k])[1]) for k in NC6 ]

4-element Vector{String}:
 "C2 x S4"
 "C2 x S4"
 "C2 x S4"
 "S3"